# Performance Prediction

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [55]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import perf_predict as pp
from layer import BaseLayer, ConvLayer, STD, DPT, PNT, MatmulLayer
from net import VGG16
from design import BasicDesign, DesignParams

## VGG-16

In [37]:
VGG16

   conv1_1: <224, 224,    3,   64, 3,   standard>
   conv1_2: <224, 224,   64,   64, 3,   standard>
   conv2_1: <112, 112,   64,  128, 3,   standard>
   conv2_2: <112, 112,  128,  128, 3,   standard>
   conv3_1: < 56,  56,  128,  256, 3,   standard>
   conv3_2: < 56,  56,  256,  256, 3,   standard>
   conv3_3: < 56,  56,  256,  256, 3,   standard>
   conv4_1: < 28,  28,  256,  512, 3,   standard>
   conv4_2: < 28,  28,  512,  512, 3,   standard>
   conv4_3: < 28,  28,  512,  512, 3,   standard>
   conv5_1: < 14,  14,  512,  512, 3,   standard>
   conv5_2: < 14,  14,  512,  512, 3,   standard>
   conv5_3: < 14,  14,  512,  512, 3,   standard>
       fc6: <25088, 4096>
       fc7: <4096, 4096>
       fc8: <4096, 1000>

In [38]:
VGG16.num_params * 1e-6

138.34412799999998

In [40]:
VGG16.num_ops * 1e-9

30.940528640000004

In [57]:
design = BasicDesign(DesignParams())
predictor = pp.BasicPredictor()
predictor.predict(VGG16, design)

(1828814848, 9.14407424, 3.3836698858647942)

In [59]:
design.resource

<     0,      0,      1,      8>